In [1]:
from torch import nn, optim
import os
import shapely
import pandas as pd
import geopandas as gpd
import numpy as np
from numpy.random import default_rng
import torch
from sklearn.metrics import recall_score, precision_score, accuracy_score, f1_score, roc_auc_score
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
import libpysal
import matplotlib.pyplot as plt

In [2]:
def load_data(year):
    os.getcwd()
    df = pd.DataFrame()
    path = "../Data/filled/" + str(year) + "/"
    for filename in os.listdir(path):
        df1 = pd.read_csv(path + filename)
        if df1.geometry.isna().any():
            print(filename)
        df = pd.concat([df, df1])
    df = gpd.GeoDataFrame(df)
    df.geometry = df.geometry.apply(shapely.wkt.loads)
    
    df = df.reset_index()
    df = df.drop(["Unnamed: 0", "index"], axis = 1)
    return df

In [3]:
def create_CNN_samples(grid, block, dims = 39):
    
    nonzero = np.transpose(grid[:,:,-2].nonzero()) # Get indices of nonzero componetns
    size = nonzero.shape[0]
    width = block * 2 + 1 # calculate widht and height. Needed later on
    
    X = np.zeros((size, width, width, dims))
    Y = np.zeros(size)
    ID = np.zeros(size)
    Y_1 = np.zeros(size)
    
    for idx, i in enumerate(nonzero):
        x, ID[idx], Y[idx], Y_1[idx] = get_neighbor_grid(grid, i, block)
        X[idx] = x.reshape(width,width, dims)
        
    X = np.moveaxis(X, -1, 1) # order the indices correctly to make sure it works in CNN
    X = torch.from_numpy(X).float()
    Y = torch.from_numpy(Y).float()
    
    return X,ID,Y, Y_1

In [4]:
def get_neighbor_grid(full, hw, block = 1):
    
    # get the nonzero (built) blocks by checking if they have a ID

    h = hw[0]
    w = hw[1]
    
    y = full[h,w,-1]
    ID = full[h,w,-2]
    Y_1_train = full[h,w,-3]
    
    hu = h - block
    hd = h + block
    hshort, hextra, wshort, wextra = 0,0,0,0
    if hu < 0:
        hshort = 0 - hu
        hu = 0
    if hd >= full.shape[0]:
        hextra = (hd - full.shape[0]) + 1
        hd = full.shape[0]

    wr = w + block
    wl = w - block

    if wr >= full.shape[1]:
        wextra = (wr - full.shape[1]) + 1
        wr = full.shape[1]
    if wl < 0:
        wshort = 0 - wl
        wl = 0

    nb = full[hu : hd + 1, wl : wr + 1, :]
    nb = np.pad(nb, ((hshort, hextra), (wshort, wextra), (0,0)), mode = "constant", constant_values = 0)
    return nb[:,:,:-3], ID, y, Y_1_train


In [5]:
import torch
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = "cpu"
print("Device: {}".format(device))

Device: cpu


In [6]:
df15 = load_data(2015).set_index("C28992R100")
df16 = load_data(2016).set_index("C28992R100")
df17 = load_data(2017).set_index("C28992R100")
df18 = load_data(2018).set_index("C28992R100")
df19 = load_data(2019).set_index("C28992R100")


In [7]:
# Get data that is available in all years
overlapping1517 = df17.index[df17.index.isin(df16.index[df16.index.isin(df15.index)])]
overlapping1518 = df18.index[df18.index.isin(overlapping1517)]
overlapping1519 = df19.index[df19.index.isin(overlapping1518)]




In [8]:
import pickle
with open("cbs_id_koppel.pickle", "rb") as f:
    a = pickle.load(f)
overlap_num = [a[x] for x in overlapping1519]

In [9]:
def get_cnn_time(year, overlap_num):
    X_train = []
    Y_train = []
    ID_train = []
    Y_1_train = []
    for filename in os.listdir("../Data/filled/grids/" + str(year) + "/"):
        n = np.load("../Data/filled/grids/" + str(year) + "/" + filename)
        X, ID, Y, Y_1 = create_CNN_samples(n, 10)
        X_train.append(X)
        Y_train.append(Y)
        ID_train.append(ID)
        Y_1_train.append(Y_1)

    Y = np.concatenate(Y_train)
    ID = np.concatenate(ID_train)
    X = np.concatenate(X_train)
    Y1 = np.concatenate(Y_1_train)
    overlap = np.isin(ID, overlap_num)
    return X[overlap], Y[overlap], ID[overlap], Y1[overlap]

    
# X15c, Y15c, ID15c, Y115c = get_cnn_time(2015, overlap_num)
# X16c, Y16c, ID16c, Y116c = get_cnn_time(2016, overlap_num)
# X17c, Y17c, ID17c, Y117c = get_cnn_time(2017, overlap_num)
# X18c, Y18c, ID18c, Y118c = get_cnn_time(2018, overlap_num)
# X19c, Y19c, ID19c, Y119c = get_cnn_time(2019, overlap_num)


In [10]:
# Fill X and Y
block_size = get_cnn_time(2015, overlap_num)[0].shape[-1]
X = np.zeros((len(overlapping1519), 5, 39, block_size, block_size))
Y = np.zeros((len(overlapping1519), 5))
Y_1 = np.zeros((len(overlapping1519), 5))
ID = np.zeros((len(overlapping1519), 5))
ss = StandardScaler()


for i, year in enumerate([2015, 2016, 2017, 2018, 2019]):
    X[:,i], Y[:,i], ID[:,i], Y_1[:,i] = get_cnn_time(year, overlap_num)

In [11]:
# Create neighbor lists for all years

df15 = df15.loc[overlapping1519]
df16 = df16.loc[overlapping1519]
df17 = df17.loc[overlapping1519]
df18 = df18.loc[overlapping1519]
df19 = df19.loc[overlapping1519]


w15 = libpysal.weights.DistanceBand.from_dataframe(df15.reset_index(), threshold=150, binary = True, silence_warnings = True)
w16 = libpysal.weights.DistanceBand.from_dataframe(df16.reset_index(), threshold=150, binary = True, silence_warnings = True)
w17 = libpysal.weights.DistanceBand.from_dataframe(df17.reset_index(), threshold=150, binary = True, silence_warnings = True)
w18 = libpysal.weights.DistanceBand.from_dataframe(df18.reset_index(), threshold=150, binary = True, silence_warnings = True)
w19 = libpysal.weights.DistanceBand.from_dataframe(df19.reset_index(), threshold=150, binary = True, silence_warnings = True)

In [12]:
# create Y(t-1) for all years

def get_y_1(y1, w):
    neighbors = [w.neighbors[x] for x in w.neighbors]
    y1 = np.array([y1[x].sum() for x in neighbors])
    y1[np.where(y1 == 0)[0]] = 0.5

    return y1

Y1_15 = get_y_1(Y_1[:,0], w15)
Y1_16 = get_y_1(Y_1[:,1], w16)
Y1_17 = get_y_1(Y_1[:,2], w17)
Y1_18 = get_y_1(Y_1[:,3], w18)
Y1_19 = get_y_1(Y_1[:,4], w19)



In [13]:
# X train is the first three years
X_train = X[:,:3]
Y_train = Y[:,2]

X_train = np.moveaxis(X_train, 2 ,-1)
X_train = X_train.reshape(-1, 39)
X_train = ss.fit_transform(X_train)
X_train = np.append(X_train.reshape(Y1_17.shape[0], -1), Y1_17.reshape(-1,1), axis = 1)

In [14]:
smote = SMOTE()
X_train_resample,Y_train_resample = smote.fit_resample(X_train,Y_train)

Y1_17r = X_train_resample[:,-1]
Y1_17r[Y1_17r>0.5] =  np.ceil(Y1_17r[Y1_17r > 0.5])

X_train_resample = np.delete(X_train_resample, -1, -1)
X_train_resample = X_train_resample.reshape(X_train_resample.shape[0], 3, block_size, block_size, 39)
X_train_resample = np.moveaxis(X_train_resample, -1, 2)

In [15]:
X_val = X[:,:4]
Y_val = Y[:,3]
X_val = np.moveaxis(X_val, 2, -1)
X_val = X_val.reshape(-1, 39)
X_val = ss.transform(X_val)
X_val = X_val.reshape(Y_val.shape[0], 4, block_size, block_size, 39)
X_val = np.moveaxis(X_val, -1, 2)



In [16]:
rng = default_rng()
def get_batch(X, Y, Y1, batch_size = 32):
    idxs = rng.integers(len(X), size = batch_size)
    return torch.tensor(X[idxs]).float().to(device), torch.tensor(Y[idxs]).float().to(device), torch.tensor(Y1[idxs]).float().to(device)



In [17]:
class RNNCNN(nn.Module):
    def __init__(self):
        super(RNNCNN, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels= 39, out_channels = 15, kernel_size = (4,4)),
            nn.MaxPool2d(2),
            nn.ReLU(), 
            nn.Dropout2d(0.3),
            nn.Conv2d(in_channels = 15, out_channels = 20, kernel_size = (4,4)),
            nn.MaxPool2d(2),
            nn.ReLU(),
            nn.Dropout2d(0.3),
            nn.Flatten()
        )
        self.GRU = nn.GRU(input_size = 219 ,hidden_size = 80, batch_first = True)
        self.net = nn.Sequential(
            nn.Linear(80,1)
        )
        
    def forward(self, X, Y1):
        orig = X[:,:,:,10,10] # original cells are at the center of 11*11 point
        
        batch_size, timesteps, C, H, W = X.size()
        
        c_in = X.view(batch_size * timesteps, C, H, W)
        c_out = self.conv(c_in)
        
        # reshape and concatenate the original data
        r_in = c_out.view(batch_size, timesteps, -1)
        r_in = torch.cat((orig, r_in), axis = 2)
        
        h0 = torch.zeros(1, X.size(0), 80).to(device)
        
        X, _ = self.GRU(r_in, h0)
        X = X[:,-1, :].unsqueeze(1)
        X = self.net(X[:,-1])
        X = X.squeeze() * Y1
 
        
        return X


In [18]:
model = RNNCNN().to(device)

In [19]:
def train(model, X_train, Y_train, Y1_train, X_val, Y_val, Y1_val, num_epochs,lr, batch_per_e =200 ):
    optimizer = optim.RMSprop(model.parameters(), lr = lr) 
    SigBCEloss = nn.BCEWithLogitsLoss()
    train_loss = []
    train_loss_history = []
    acc_history = []
    ROC_history = []
    f1_score_history = []
    val_loss_history = []
    best_auc = 0
    
    for epoch in range(num_epochs):
        epoch_loss = []
        print("epoch: {} of {}".format(epoch, num_epochs))
        for batch in range(batch_per_e):
            x, y, y1 = get_batch(X_train, Y_train, Y1_train, 6)
        
            model.train()

            optimizer.zero_grad()
            out = model(x, y1).squeeze()

            
            loss = SigBCEloss(out, y)
            loss.backward()
            optimizer.step()
            

            epoch_loss.append(loss.cpu().detach().numpy())
                
        model.eval()

        out = np.zeros(len(X_val))
        c_1 = 0
        for c in range(1000, len(X_val) + 1000, 1000):
            
            if c >= len(X_val):
                c = len(X_val) - 1
            x = torch.tensor(X_val[c_1 : c]).float().to(device)
            y1 = torch.tensor(Y1_val[c_1 : c]).float().to(device)
            

            temp_out = model(x, y1).cpu().detach().numpy()
            out[c_1 : c] = temp_out
            c_1 = c
            
            
        val_loss = SigBCEloss(torch.tensor(out).float(), torch.tensor(Y_val).float())
        
        
        preds = np.zeros(len(out))
        pos = out.argsort()[-((Y_val == 1).sum()):]
        preds[pos] = 1

        
        acc = accuracy_score(Y_val, preds)
        ROC = roc_auc_score(Y_val, out)
        if ROC > best_auc:
            print(ROC)
            bets_auc = ROC
        
        f1 = f1_score(Y_val, preds)

        acc_history.append(acc)
        ROC_history.append(ROC)
        train_loss_history.append(np.sum(epoch_loss) / batch_per_e)
        f1_score_history.append(f1)
        val_loss_history.append(val_loss.detach().numpy())

        
        train_loss = []
    print(out)
    result = np.argmax(ROC_history)
    print("best auc: {}, f1: {}, epoch: {}".format(ROC_history[result], f1_score_history[result], result))
    return acc_history, ROC_history, train_loss_history, f1_score_history, val_loss_history

In [20]:
n_epochs = 500 
hists = train(model, X_train_resample, Y_train_resample, Y1_17r, X_val, Y_val, Y1_18, n_epochs,0.00000051)

epoch: 0 of 500
0.5041148531027496
epoch: 1 of 500
0.5101431071015796
epoch: 2 of 500
0.5170438087696018
epoch: 3 of 500
0.5240823826748284
epoch: 4 of 500
0.5306164208866629
epoch: 5 of 500
0.538630802048999
epoch: 6 of 500
0.5464219211506356
epoch: 7 of 500
0.5543231775180857
epoch: 8 of 500
0.5611610518956095
epoch: 9 of 500
0.5672898563132286
epoch: 10 of 500
0.5727089219211506
epoch: 11 of 500
0.5787251366237176
epoch: 12 of 500
0.583310280310489
epoch: 13 of 500
0.5877687884365681
epoch: 14 of 500
0.591127306194084
epoch: 15 of 500
0.5955214709701354
epoch: 16 of 500
0.5988187220866686
epoch: 17 of 500
0.6012153446627571
epoch: 18 of 500
0.6042917861678296
epoch: 19 of 500
0.6068119230055791
epoch: 20 of 500
0.6103392584506941
epoch: 21 of 500
0.6121801114392934
epoch: 22 of 500
0.6140493236590899
epoch: 23 of 500
0.6164644064894482
epoch: 24 of 500
0.6187636445357647
epoch: 25 of 500
0.6205253948889174
epoch: 26 of 500
0.6222189225632465
epoch: 27 of 500
0.6236794229699072
epoch

KeyboardInterrupt: 

In [ ]:
plt.plot(hists[-3], alpha = 0.5)
plt.hlines(np.mean(hists[-3]), 0, n_epochs, color = "r")
plt.plot(hists[-1], alpha = 0.5, color = "green")

In [ ]:
print(np.max(hists[-4]), np.argmax(hists[-4]))
plt.plot(hists[-4], alpha = 0.5)
plt.hlines(np.mean(hists[-4]), 0, n_epochs, color = "r")

X_val


In [ ]:
for c in range(0, len(X_val) + 1000, 1000):
    print(c)

In [ ]:
len(X_val)